# Clustering Crypto

In [36]:
# Initial imports
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Fetching Cryptocurrency Data

In [37]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"

coinlist_json = requests.get(url).json()

In [38]:
# Create a DataFrame 
# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.

crypto_df = pd.DataFrame(data = coinlist_json['Data']).T
crypto_df.head(3)

,Algorithm,BlockNumber,BlockReward,BlockTime,BuiltOn,CoinName,ContentCreatedOn,DecimalPlaces,FullName,FullyPremined,...,Rating,SmartContractAddress,SortOrder,Sponsored,Symbol,Taxonomy,TotalCoinSupply,TotalCoinsFreeFloat,TotalCoinsMined,Url
42,Scrypt,195131,0,0,N/A,42 Coin,1427211129,0,42 Coin (42),0,...,"{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",N/A,34,False,42,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...",42,N/A,42,/coins/42/overview
300,N/A,0,0,0,7605,300 token,1517935016,18,300 token (300),0,...,"{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",0xaec98a708810414878c3bcdf46aad31ded4a4557,2212,False,300,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...",300,N/A,300,/coins/300/overview
365,X11,NaN,NaN,NaN,N/A,365Coin,1480032918,0,365Coin (365),0,...,"{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",N/A,916,False,365,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...",2300000000,N/A,NaN,/coins/365/overview


In [39]:
# Alternatively, use the provided csv file:
# file_path = Path("Resources/crypto_data.csv")
# Create a DataFrame

### Data Preprocessing

In [40]:
# Keep only necessary columns: 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
crypto_df = crypto_df[['CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply']]
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,42,42
300,300 token,N/A,True,N/A,300,300
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.26594e+06,532000000
433,433 Token,N/A,False,N/A,1.12518e+08,1000000000


In [41]:
# Keep only cryptocurrencies that are trading
crypto_df = crypto_df[crypto_df['IsTrading'] == True]
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,42,42
300,300 token,N/A,True,N/A,300,300
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.26594e+06,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000


In [42]:
# Keep only cryptocurrencies with a working algorithm
crypto_df = crypto_df[crypto_df['Algorithm'] != 'N/A']
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,42,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.26594e+06,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0,0


In [43]:
# Remove the "IsTrading" column
crypto_df.drop(columns=['IsTrading'], inplace=True)
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,42,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.26594e+06,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0,0


In [44]:
# Remove rows with at least 1 null value
crypto_df.dropna(inplace=True)
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,42,42
404,404Coin,Scrypt,PoW/PoS,1.26594e+06,532000000
808,808,SHA-256,PoW/PoS,0,0
1337,EliteCoin,X13,PoW/PoS,2.94803e+10,314159265359
BTCD,BitcoinDark,SHA-256,PoW/PoS,1288862,22000000


In [45]:
# Remove rows with cryptocurrencies having no coins mined
crypto_df = crypto_df[crypto_df['TotalCoinsMined'] > 0]
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,42,42
404,404Coin,Scrypt,PoW/PoS,1.26594e+06,532000000
1337,EliteCoin,X13,PoW/PoS,2.94803e+10,314159265359
BTCD,BitcoinDark,SHA-256,PoW/PoS,1288862,22000000
XPY,PayCoin,SHA-256,PoS,1.19953e+07,12500000


In [46]:
# Drop rows where there are 'N/A' text values
crypto_df = crypto_df[
    (crypto_df['Algorithm'] != 'N/A') & 
    (crypto_df['ProofType'] != 'N/A') &
    (crypto_df['TotalCoinsMined'] != 'N/A') & 
    (crypto_df['TotalCoinSupply'] != 'N/A')]

crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,42,42
404,404Coin,Scrypt,PoW/PoS,1.26594e+06,532000000
1337,EliteCoin,X13,PoW/PoS,2.94803e+10,314159265359
BTCD,BitcoinDark,SHA-256,PoW/PoS,1288862,22000000
XPY,PayCoin,SHA-256,PoS,1.19953e+07,12500000


In [47]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
coinName_df = pd.DataFrame({'CoinName': crypto_df['CoinName']}) 
coinName_df.head()

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTCD,BitcoinDark
XPY,PayCoin


In [48]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_df.drop(columns=['CoinName'], inplace=True)
crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,42,42
404,Scrypt,PoW/PoS,1.26594e+06,532000000
1337,X13,PoW/PoS,2.94803e+10,314159265359
BTCD,SHA-256,PoW/PoS,1288862,22000000
XPY,SHA-256,PoS,1.19953e+07,12500000


In [49]:
# Create dummy variables for text features
crypto_df_encoded = pd.get_dummies(crypto_df, columns=['Algorithm','ProofType'])
crypto_df_encoded.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_BMW512 / Echo512,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,42,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.26594e+06,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.94803e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTCD,1288862,22000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
XPY,1.19953e+07,12500000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
# Standardize data
crypto_scaled = StandardScaler().fit_transform(crypto_df_encoded)
# print(crypto_scaled[0:5])

### Reducing Dimensions Using PCA

In [51]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)

In [52]:
# Create a DataFrame with the principal components data
crypto_pca = pca.fit_transform(crypto_scaled)
# crypto_pca = pca.fit_transform(crypto_df_encoded)

crypto_pca_df = pd.DataFrame(
    data=crypto_pca, 
    columns=["principal component 1", "principal component 2", "principal component 3"]    
)
crypto_pca_df.head()

,principal component 1,principal component 2,principal component 3
0,-0.351310,0.950061,-0.581437
1,-0.347432,0.949973,-0.581450
2,2.337355,1.695733,-0.750330
3,-0.338475,0.809417,-0.466555
4,-0.296987,0.619342,-0.143497


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [53]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(crypto_pca_df)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)

:Curve   [k]   (inertia)

Running K-Means with `k=<your best value for k here>`

In [54]:
# Initialize the K-Means model
model = KMeans(n_clusters=5, random_state=5)

# Fit the model
model.fit(crypto_pca_df)

# Predict clusters
predictions = model.predict(crypto_pca_df)
# print(predictions)

# Create a new DataFrame including predicted clusters and cryptocurrencies features
crypto_pca_df["class"] = model.labels_
crypto_pca_df.head()

,principal component 1,principal component 2,principal component 3,class
0,-0.351310,0.950061,-0.581437,0
1,-0.347432,0.949973,-0.581450,0
2,2.337355,1.695733,-0.750330,4
3,-0.338475,0.809417,-0.466555,0
4,-0.296987,0.619342,-0.143497,0


### Visualizing Results

#### 3D-Scatter with Clusters

In [55]:
# Create a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    crypto_pca_df,
    x="principal component 1",
    y="principal component 2",
    z="principal component 3",
    color="class",
    symbol="class",
    size="class",
    width=800,
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

#### Table of Tradable Cryptocurrencies

In [56]:
# Table with tradable cryptos
display(crypto_df) 

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,42,42
404,Scrypt,PoW/PoS,1.26594e+06,532000000
1337,X13,PoW/PoS,2.94803e+10,314159265359
BTCD,SHA-256,PoW/PoS,1288862,22000000
XPY,SHA-256,PoS,1.19953e+07,12500000
PRC,Scrypt,PoW,6268245,21000000
KOBO,X15,PoW/PoS,2.55844e+07,350000000
ARG,Scrypt,PoW,1.38262e+07,64000000
BLU,Scrypt,PoW/PoS,647272520,0
XMY,Multiple,PoW,1747693250,2000000000


In [57]:
# Print the total number of tradable cryptocurrencies
num_of_tradable_cryptocurrencies = len(crypto_df)

print(num_of_tradable_cryptocurrencies)

540


#### Scatter Plot with Tradable Cryptocurrencies

In [58]:
# Scale data to create the scatter plot
housing_df_min_max_scale =  pd.DataFrame(MinMaxScaler().fit_transform(crypto_df_encoded))
# housing_df_min_max_scale

In [59]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
crypto_df.hvplot.scatter(x="TotalCoinsMined", y="TotalCoinSupply")

:Scatter   [TotalCoinsMined]   (TotalCoinSupply)